In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    #model_name = "unsloth/codellama-13b-bnb-4bit",
    model_name = "cognitivecomputations/Dolphin3.0-Llama3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.6.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
import json
import random
from typing import Dict, List, Any
from pathlib import Path

alpaca_prompt_backup = """You must strictly answer questions ONLY based on the provided context.
If the context does NOT include the relevant information, your answer must be exactly:
"Don't know" or "Not provided in context".
Never guess or add any information not mentioned in the context.

### Question:
{}

### Context:
{}

### Answer:
{}"""

def format_prompt(qa: Dict[str, str]) -> str:
    if qa["instruction"].startswith("[Context QA]"):
        return f"""You must strictly answer questions ONLY based on the provided context.
If the context does NOT include the relevant information, your answer must be exactly:
"Don't know" or "Not provided in context".
Never guess or add any information not mentioned in the context.

### Question:
{qa['instruction'].replace('[Context QA] ', '')}

### Context:
{qa['input']}

### Answer:
{qa['output']}"""

    elif qa["instruction"].startswith("[General QA]"):
        return f"""Answer the following question to the best of your knowledge and reasoning.
There is no context provided — you may rely on general understanding.

### Question:
{qa['instruction'].replace('[General QA] ', '')}

### Answer:
{qa['output']}"""

    else:
        raise ValueError("Unknown instruction label: must start with [Context QA] or [General QA]")


import json

required_fields = {"instruction", "input", "output"}

input_path = "/content/drive/MyDrive/Colab Notebooks/docs/dataset_claude_raw.jsonl"
output_clean = "/content/drive/MyDrive/Colab Notebooks/docs/dataset_clean.jsonl"
output_invalid = "/content/drive/MyDrive/Colab Notebooks/docs/dataset_invalid.jsonl"

clean_count = 0
invalid_count = 0

with open(input_path, "r", encoding="utf-8",errors='ignore') as infile, \
     open(output_clean, "w", encoding="utf-8") as cleanfile, \
     open(output_invalid, "w", encoding="utf-8") as invalidfile:

    for i, line in enumerate(infile, 1):
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
            if not required_fields.issubset(obj):
                missing = required_fields - obj.keys()
                raise ValueError(f"Missing fields: {', '.join(missing)}")
            json.dump(obj, cleanfile, ensure_ascii=False)
            cleanfile.write("\n")
            clean_count += 1
        except (json.JSONDecodeError, ValueError) as e:
            print(f"[Line {i}] Invalid: {e}")
            invalidfile.write(line + "\n")
            invalid_count += 1

print(f"\n✅ Cleaned entries written: {clean_count}")
print(f"⚠️  Invalid entries saved: {invalid_count}")



EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    questions = examples["instruction"]
    contexts       = examples["input"]
    answers      = examples["output"]
    texts = []
    for question, context, answer in zip(questions, contexts, answers):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt_backup.format(question, context, answer) + EOS_TOKEN
        #qa_dict = {
        #    "instruction": question,
        #    "input": context,
        #    "output": answer
        #}
        #text = format_prompt(qa_dict) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Colab Notebooks/docs/dataset_clean.jsonl", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset[0]["text"])


✅ Cleaned entries written: 2044
⚠️  Invalid entries saved: 0


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2044 [00:00<?, ? examples/s]

You must strictly answer questions ONLY based on the provided context.
If the context does NOT include the relevant information, your answer must be exactly:
"Don't know" or "Not provided in context".
Never guess or add any information not mentioned in the context.

### Question:
How do I specify multiple item-IDs when working with a file reference?

### Context:
Item list syntax: command file.reference item-ID 
 command file.reference item-ID item-ID item-ID...
 command file.reference{*}

The item list specifies one or more item-IDs in the file defined by the associated file reference. If a select list is not active, a null item-ID implies a new item for Update processor and all items for the other processors. Any command requiring a select list can obtain it from a previously selected list. To cause a processor to use the select list, the item list must be null. An item-ID with the same name as a language element in either the master dictionary or the dictionary of the file, must be 

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    #formatting_func = formatting_prompts_func, # Add this line
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/2044 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.135 GB of memory reserved.


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,044 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.722500
2,2.576500
3,2.613000
4,2.456600
5,2.297800
6,2.060900
7,1.897200
8,1.719800
9,1.515000
10,1.629100


In [8]:
model.save_pretrained("/content/drive/MyDrive/Llama3.1-8b-instruct-lora")  # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Llama3.1-8b-instruct-lora")

('/content/drive/MyDrive/Llama3.1-8b-instruct-lora/tokenizer_config.json',
 '/content/drive/MyDrive/Llama3.1-8b-instruct-lora/special_tokens_map.json',
 '/content/drive/MyDrive/Llama3.1-8b-instruct-lora/chat_template.jinja',
 '/content/drive/MyDrive/Llama3.1-8b-instruct-lora/tokenizer.json')

Work with RAG which do a good chunk

In [5]:
!pip install faiss-gpu
!pip install faiss-cpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 23.8 MB/s eta 0:00:00


In [6]:
import json
import numpy as np
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass
import faiss
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from unsloth import FastLanguageModel
from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseMessage, HumanMessage, AIMessage

@dataclass
class RetrievedDocument:
    """Represents a retrieved document with relevance score"""
    doc_id: str
    title: str
    content: str
    syntax: str
    examples: List[str]
    options: str
    metadata: Dict
    relevance_score: float
    retrieval_type: str

class AlignedRAGSystem:
    """RAG system aligned with SFT training format"""

    def __init__(self,
                 embedding_model_name: str = "all-MiniLM-L6-v2",
                 sft_model_path: str = None,
                 lora_adapter_path: str = None):
        """Initialize the aligned RAG system"""

        # Initialize embedding model
        self.embedding_model = SentenceTransformer(embedding_model_name)

        # Initialize document storage and indexes
        self.documents = []
        self.indexes = {}
        self.embeddings = {}

        # Initialize SFT model (your fine-tuned model)
        self.tokenizer = None
        self.model = None
        self.sft_model_path = sft_model_path
        self.lora_adapter_path = lora_adapter_path

        # Initialize conversation memory
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="answer"
        )

        # Aligned prompt template (matches your SFT training)
        self.aligned_prompt_template = """You must strictly answer questions ONLY based on the provided context. If the context does NOT include the relevant information, your answer must be exactly: "Don't know" or "Not provided in context". Never guess or add any information not mentioned in the context.

### Question: {question}

### Context: {context}

### Answer: """

    def load_sft_model(self):
      """Load your fine-tuned SFT model with LoRA adapter"""
      try:
          print("Loading SFT model with LoRA adapter...")

          # Method 1: Load the LoRA adapter directly (Recommended)
          if self.lora_adapter_path:
              self.model, self.tokenizer = FastLanguageModel.from_pretrained(
                  model_name=self.lora_adapter_path,  # Load directly from LoRA path
                  max_seq_length=2048,
                  dtype=None,
                  load_in_4bit=True,
              )
          else:
              # Load base model if no LoRA adapter specified
              self.model, self.tokenizer = FastLanguageModel.from_pretrained(
                  model_name=self.sft_model_path,
                  max_seq_length=2048,
                  dtype=None,
                  load_in_4bit=True,
              )

          # Set to inference mode
          FastLanguageModel.for_inference(self.model)

          print("SFT model loaded successfully!")

      except Exception as e:
          print(f"Error loading SFT model with Unsloth: {e}")
          print("Trying alternative loading method...")

          try:
              # Alternative method using transformers + peft
              from peft import PeftModel

              # Load base model
              self.tokenizer = AutoTokenizer.from_pretrained(self.sft_model_path)
              base_model = AutoModelForCausalLM.from_pretrained(
                  self.sft_model_path,
                  torch_dtype=torch.float16,
                  device_map="auto"
              )

              # Load LoRA adapter if specified
              if self.lora_adapter_path:
                  self.model = PeftModel.from_pretrained(base_model, self.lora_adapter_path)
                  print("LoRA adapter loaded successfully!")
              else:
                  self.model = base_model

          except Exception as e2:
              print(f"Error with alternative loading method: {e2}")
              print("Falling back to base model only...")

              # Final fallback - base model only
              self.tokenizer = AutoTokenizer.from_pretrained(self.sft_model_path)
              self.model = AutoModelForCausalLM.from_pretrained(
                  self.sft_model_path,
                  torch_dtype=torch.float16,
                  device_map="auto"
              )

    def load_documents(self, jsonl_path: str):
        """Load documents from JSONL file"""
        print(f"Loading documents from {jsonl_path}")
        self.documents = []

        with open(jsonl_path, 'r', encoding='utf-8') as f:
            for line in f:
                doc = json.loads(line.strip())
                self.documents.append(doc)

        print(f"Loaded {len(self.documents)} documents")
        return len(self.documents)

    def create_embeddings_hybrid(self):
        """Create hybrid embeddings (both combined and separate)"""
        print("Creating hybrid embeddings...")

        # Combined embeddings
        combined_texts = []
        for doc in self.documents:
            parts = []

            if doc['content']:
                parts.append(f"Content: {doc['content']}")
            if doc['syntax']:
                parts.append(f"Syntax: {doc['syntax']}")
            if doc['examples']:
                examples_text = " | ".join(doc['examples'])
                parts.append(f"Examples: {examples_text}")
            if doc['options']:
                parts.append(f"Options: {doc['options']}")

            combined_text = " | ".join(parts)
            combined_texts.append(combined_text)

        # Create combined embeddings
        embeddings = self.embedding_model.encode(combined_texts, convert_to_tensor=False)
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatIP(dimension)
        faiss.normalize_L2(embeddings)
        index.add(embeddings.astype(np.float32))

        self.indexes['combined'] = index
        self.embeddings['combined'] = embeddings

        # Separate embeddings for different content types
        content_types = ['content', 'syntax', 'examples', 'options']

        for content_type in content_types:
            texts = []
            doc_indices = []

            for i, doc in enumerate(self.documents):
                text = ""

                if content_type == 'content' and doc['content']:
                    text = doc['content']
                elif content_type == 'syntax' and doc['syntax']:
                    text = doc['syntax']
                elif content_type == 'examples' and doc['examples']:
                    text = " | ".join(doc['examples'])
                elif content_type == 'options' and doc['options']:
                    text = doc['options']

                if text.strip():
                    texts.append(text)
                    doc_indices.append(i)

            if texts:
                embeddings = self.embedding_model.encode(texts, convert_to_tensor=False)
                dimension = embeddings.shape[1]
                index = faiss.IndexFlatIP(dimension)
                faiss.normalize_L2(embeddings)
                index.add(embeddings.astype(np.float32))

                self.indexes[content_type] = index
                self.embeddings[content_type] = embeddings
                setattr(self, f'{content_type}_doc_indices', doc_indices)

                print(f"Created {content_type} index with {len(embeddings)} entries")

    def intelligent_search(self, query: str, top_k: int = 5) -> List[RetrievedDocument]:
        """Intelligent search that determines the best search strategy"""
        query_lower = query.lower()

        # Query classification keywords
        syntax_keywords = ['syntax', 'how to use', 'format', 'parameters', 'arguments']
        example_keywords = ['example', 'sample', 'demo', 'show me', 'illustration']
        options_keywords = ['options', 'flags', 'settings', 'configuration']

        # Search strategy based on query intent
        if any(keyword in query_lower for keyword in syntax_keywords):
            return self._prioritized_search(query, 'syntax', top_k)
        elif any(keyword in query_lower for keyword in example_keywords):
            return self._prioritized_search(query, 'examples', top_k)
        elif any(keyword in query_lower for keyword in options_keywords):
            return self._prioritized_search(query, 'options', top_k)
        else:
            return self._combined_search(query, top_k)

    def _prioritized_search(self, query: str, content_type: str, top_k: int) -> List[RetrievedDocument]:
        """Search with priority for specific content type"""
        results = []

        # First search in specific content type
        if content_type in self.indexes:
            type_results = self._search_by_type(query, content_type, top_k//2)
            results.extend(type_results)

        # Then search in combined index
        if 'combined' in self.indexes:
            combined_results = self._combined_search(query, top_k//2)
            results.extend(combined_results)

        # Deduplicate and return top_k
        return self._deduplicate_results(results)[:top_k]

    def _search_by_type(self, query: str, content_type: str, top_k: int) -> List[RetrievedDocument]:
        """Search in specific content type"""
        if content_type not in self.indexes:
            return []

        query_embedding = self.embedding_model.encode([query], convert_to_tensor=False)
        faiss.normalize_L2(query_embedding)

        scores, indices = self.indexes[content_type].search(query_embedding.astype(np.float32), top_k)
        doc_indices = getattr(self, f'{content_type}_doc_indices')

        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx < len(doc_indices):
                doc_idx = doc_indices[idx]
                doc = self.documents[doc_idx]
                results.append(self._create_retrieved_doc(doc, float(score), content_type))

        return results

    def _combined_search(self, query: str, top_k: int) -> List[RetrievedDocument]:
        """Search using combined embeddings"""
        if 'combined' not in self.indexes:
            return []

        query_embedding = self.embedding_model.encode([query], convert_to_tensor=False)
        faiss.normalize_L2(query_embedding)

        scores, indices = self.indexes['combined'].search(query_embedding.astype(np.float32), top_k)

        results = []
        for score, idx in zip(scores[0], indices[0]):
            doc = self.documents[idx]
            results.append(self._create_retrieved_doc(doc, float(score), 'combined'))

        return results

    def _create_retrieved_doc(self, doc: dict, score: float, retrieval_type: str) -> RetrievedDocument:
        """Create RetrievedDocument from document dict"""
        return RetrievedDocument(
            doc_id=doc['id'],
            title=doc['title'],
            content=doc['content'],
            syntax=doc['syntax'],
            examples=doc['examples'],
            options=doc['options'],
            metadata=doc['metadata'],
            relevance_score=score,
            retrieval_type=retrieval_type
        )

    def _deduplicate_results(self, results: List[RetrievedDocument]) -> List[RetrievedDocument]:
        """Remove duplicate documents from results"""
        seen_ids = set()
        unique_results = []
        for result in results:
            if result.doc_id not in seen_ids:
                seen_ids.add(result.doc_id)
                unique_results.append(result)
        return unique_results

    def format_context_for_sft(self, retrieved_docs: List[RetrievedDocument],
                              max_context_length: int = 1500) -> str:
        """Format context exactly matching your SFT training format"""

        if not retrieved_docs:
            return "No relevant context found."

        context_parts = []
        current_length = 0

        for i, doc in enumerate(retrieved_docs):
            # Format exactly like your training data "input" field
            doc_parts = []

            if doc.syntax:
                doc_parts.append(f"Syntax: {doc.syntax}")

            if doc.content:
                doc_parts.append(f"Description: {doc.content}")

            if doc.options:
                doc_parts.append(f"Options: {doc.options}")

            if doc.examples:
                # Limit examples to avoid context overflow
                examples = doc.examples[:2]  # Take first 2 examples
                for j, example in enumerate(examples):
                    doc_parts.append(f"Example: {example}")

            # Add source information
            if doc.metadata:
                source_info = f"Source: {doc.metadata.get('section_component', 'N/A')}"
                if 'page_number' in doc.metadata:
                    source_info += f", Page: {doc.metadata['page_number']}"
                doc_parts.append(source_info)

            doc_text = "\n".join(doc_parts)

            # Check context length limit
            if current_length + len(doc_text) > max_context_length:
                break

            if context_parts:  # Add separator between documents
                context_parts.append("\n---\n")

            context_parts.append(doc_text)
            current_length += len(doc_text)

        return "".join(context_parts)

    def generate_response(self, query: str, context: str) -> str:
        """Generate response using your fine-tuned SFT model"""
        if not self.model or not self.tokenizer:
            return "Model not loaded. Please call load_sft_model() first."

        # Format prompt exactly matching your SFT training
        prompt = self.aligned_prompt_template.format(
            question=query,
            context=context
        )

        try:
            # Tokenize
            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=2048,  # Adjust based on your model
                padding=True
            )

            # Move to device
            if torch.cuda.is_available():
                inputs = {k: v.cuda() for k, v in inputs.items()}

            # Generate
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=256,  # Adjust as needed
                    temperature=0.1,      # Low temperature for factual responses
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                )

            # Decode response
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Extract only the answer part (after "### Answer:")
            if "### Answer:" in response:
                answer = response.split("### Answer:")[-1].strip()
            else:
                answer = response

            return answer

        except Exception as e:
            print(f"Error generating response: {e}")
            return "Sorry, I encountered an error generating the response."

    def chat(self, query: str, use_memory: bool = True) -> dict:
        """Main chat interface with conversation memory"""

        # Retrieve relevant documents
        retrieved_docs = self.intelligent_search(query, top_k=5)

        # Format context
        context = self.format_context_for_sft(retrieved_docs)

        # Generate response
        response = self.generate_response(query, context)

        # Update conversation memory if enabled
        if use_memory:
            self.memory.chat_memory.add_user_message(query)
            self.memory.chat_memory.add_ai_message(response)

        return {
            "question": query,
            "answer": response,
            "context": context,
            "retrieved_docs": len(retrieved_docs),
            "retrieval_types": [doc.retrieval_type for doc in retrieved_docs],
            "relevance_scores": [doc.relevance_score for doc in retrieved_docs]
        }

    def get_conversation_history(self) -> List[BaseMessage]:
        """Get conversation history"""
        return self.memory.chat_memory.messages

    def clear_memory(self):
        """Clear conversation memory"""
        self.memory.clear()

# Usage example
def main():
    # Initialize the aligned RAG system
    rag_system = AlignedRAGSystem(
        embedding_model_name="all-MiniLM-L6-v2",
        sft_model_path="cognitivecomputations/Dolphin3.0-Llama3.1-8B",  # Your base model path
        lora_adapter_path="/content/drive/MyDrive/Llama3.1-8b-instruct-lora"  # Your LoRA adapter path
    )

    # Load your fine-tuned model
    rag_system.load_sft_model()

    # Load documents
    rag_system.load_documents('/content/drive/MyDrive/Colab Notebooks/docs/merged_knowledge_segments.jsonl')

    # Create embeddings
    rag_system.create_embeddings_hybrid()

    # Test queries
    test_queries = [
        "How do I retrieve a specific value from a MultiValue set using user exits?",
        "Show me syntax for OCONV function",
        "What are the examples of string manipulation?",
        "How to debug FlashBASIC programs?"
    ]

    print("=== Testing Aligned RAG System ===")
    for query in test_queries:
        print(f"\nQuery: {query}")
        print("-" * 60)

        result = rag_system.chat(query)

        print(f"Answer: {result['answer']}")
        print(f"Retrieved docs: {result['retrieved_docs']}")
        print(f"Retrieval types: {result['retrieval_types']}")
        print(f"Relevance scores: {[f'{score:.3f}' for score in result['relevance_scores']]}")

        # Show a portion of the context
        print(f"\nContext preview:")
        print(result['context'][:300] + "..." if len(result['context']) > 300 else result['context'])

if __name__ == "__main__":
    main()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-6-1543826167>:50: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(


Loading SFT model with LoRA adapter...
==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

SFT model loaded successfully!
Loading documents from /content/drive/MyDrive/Colab Notebooks/docs/merged_knowledge_segments.jsonl
Loaded 2138 documents
Creating hybrid embeddings...
Created content index with 2115 entries
Created syntax index with 1300 entries
Created examples index with 1024 entries
Created options index with 277 entries
=== Testing Aligned RAG System ===

Query: How do I retrieve a specific value from a MultiValue set using user exits?
------------------------------------------------------------
Answer: 1
The 'uc070' user shell command returns the specified value from a multi-valued set. For example, 'uc070;3' returns the 3rd multi-value.
Retrieved docs: 5
Retrieval types: ['combined', 'combined', 'combined', 'combined', 'combined']
Relevance scores: ['0.420', '0.414', '0.367', '0.346', '0.333']

Context preview:
Syntax: Syntax
set {variable.name{={"|'}variable.value{"|'}}}
Description: modifies the value of a D
3
user shell variable.
If the "set" command is executed

Work with RAG which ONLY do a simple character chunk

In [ ]:
# 推理模板，与训练时保持一致
def format_prompt_for_inference(question: str, context: str = "") -> str:
    """用于推理阶段的 prompt 构造"""
    if context.strip():  # 有上下文 => Context QA
        prompt = f"""You must strictly answer questions ONLY based on the provided context.
If the context does NOT include the relevant information, your answer must be exactly:
"Don't know" or "Not provided in context".
Never guess or add any information not mentioned in the context.

### Question:
{question}

### Context:
{context}

### Answer:"""
    else:  # 无上下文 => General QA
        prompt = f"""Answer the following question to the best of your knowledge and reasoning.
There is no context provided — you may rely on general understanding.

### Question:
{question}

### Answer:"""
    return prompt

# 启用快速推理
model = FastLanguageModel.for_inference(model)

# 示例使用
def get_model_response(question: str, context: str = "") -> str:
    # 创建推理输入
    #prompt = create_inference_prompt(question, context)
    #prompt = format_prompt_for_inference(question, context)
    # tokenize输入
    inputs = tokenizer(
        [alpaca_prompt_backup.format(question,
                              context,
                              "",
        )
        ],return_tensors="pt"
    ).to("cuda")
    #inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    # 生成回答
    outputs = model.generate(
        **inputs,
        max_new_tokens=64,
        use_cache=True,
        temperature=0.7,  # 可以调整以控制输出的创造性
        do_sample=True,   # 使用采样而不是贪婪解码
    )

    # 解码输出
    response = tokenizer.batch_decode(outputs)[0]

    # 清理响应（移除prompt部分）
    response = response.split("Answer:")[-1].strip()

    return response

# 使用示例
if __name__ == "__main__":
    # 示例1：有上下文的查询
    question = "Can you provide the syntax for check-sum command?"
    context = """name: check-sum
Syntax: check-sum file.reference {item.list} {selection.criteria} {modifiers} {(options)}
description: The check-sum command calculates and displays a checksum for the specified item."""

    response = get_model_response(question, context)
    print("\n=== Example 1: With Context ===")
    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"Response: {response}")

    # 示例2：无上下文的查询
    question = "How do I enable or disable x-on/x-off flow control on a port?"
    response = get_model_response(question)
    print("\n=== Example 2: Without Context ===")
    print(f"Question: {question}")
    print(f"Response: {response}")


=== Example 1: With Context ===
Question: Can you provide the syntax for check-sum command?
Context: name: check-sum
Syntax: check-sum file.reference {item.list} {selection.criteria} {modifiers} {(options)}
description: The check-sum command calculates and displays a checksum for the specified item.
Response: Syntax: check-sum file.reference {item.list} {selection.criteria} {modifiers} {(options)}
Description: The check-sum command calculates and displays a checksum for the specified item. The file.reference is the name of the file containing the item list, and the item.list is the list of items to be checked. You

=== Example 2: Without Context ===
Question: How do I enable or disable x-on/x-off flow control on a port?
Response: Use the x-on and x-off commands to enable and disable x-on/x-off flow control on a port. When flow control is enabled, the port will automatically disable when a device goes into idle mode and re-enable when a device becomes active. This helps prevent excessi

In [ ]:
!pip install -q langchain langchain_community chromadb sentence-transformers unstructured gradio pdf2image pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 31.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from unsloth import FastLanguageModel
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
import gradio as gr
from typing import Any, List, Optional
import os

In [ ]:

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False

SAVE_PATH = "/content/drive/MyDrive/Llama3.1-8b-instruct-lora"
PDF_DIR = "/content/drive/MyDrive/Colab Notebooks/docs/"

class UnslothLLM(LLM):
    model: Any
    tokenizer: Any
    def __init__(self, model, tokenizer, **kwargs):
        super().__init__(model=model, tokenizer=tokenizer, **kwargs)
        self.model = model
        self.tokenizer = tokenizer

    def _call(self, prompt: str, **kwargs) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        outputs = self.model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            do_sample=True
        )
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Ensure that the response starts after the prompt
        if response.startswith(prompt):
            return response[len(prompt):].strip()
        return response[len(prompt):].strip()

    @property
    def _llm_type(self):
        return "custom"

# Add steps to clear GPU memory
print("Clearing GPU cache...")
torch.cuda.empty_cache()
# Optional: Delete previous model/tokenizer variables if they still exist
if 'model' in globals():
    del model
if 'tokenizer' in globals():
    del tokenizer
# Optional: Run Python garbage collection - can sometimes help
import gc
gc.collect()

# Add a check for GPU Memory after clearing
print("Checking GPU Memory after clearing cache:")
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved before loading.")

model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = SAVE_PATH, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
)
llm = UnslothLLM(model=model, tokenizer=tokenizer)

Clearing GPU cache...
Checking GPU Memory after clearing cache:
GPU = Tesla T4. Max memory = 14.741 GB.
7.43 GB of memory reserved before loading.
==((====))==  Unsloth 2025.6.1: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
!pip install -q pi_heif
!pip install -q unstructured_inference

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 73.9 MB/s eta 0:00:00


In [ ]:
!sudo apt-get update
!sudo apt-get install -y poppler-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,765 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,742 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,984 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,017 kB]
Hit:13 https://ppa.launchpadconten

In [ ]:
!pip install -q unstructured-pytesseract
!pip install -q tqdm

In [ ]:
# 1. 安装必要的依赖
!pip install -q langchain langchain_community chromadb sentence-transformers pypdf tqdm

# 2. 导入核心库
from google.colab import drive
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from tqdm.notebook import tqdm
import torch
import gc
import os
from typing import List, Dict
import logging
from datetime import datetime

# 3. 简单的云平台优化PDF处理类
class SimplePDFProcessor:
    def __init__(self, pdf_dir: str, batch_size: int = 5):
        """
        初始化PDF处理器

        Args:
            pdf_dir: PDF文件目录
            batch_size: 批处理大小
        """
        self.pdf_dir = pdf_dir
        self.batch_size = batch_size
        self.embeddings = self._init_embeddings()

    def _init_embeddings(self) -> HuggingFaceEmbeddings:
        """初始化embedding模型，使用GPU如果可用"""
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        return HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={'device': device}
        )

    def process_pdf(self, file_path: str) -> List[Dict]:
        """处理单个PDF文件，使用批处理来优化内存使用"""
        try:
            # 使用更高效的PyPDFLoader
            loader = PyPDFLoader(file_path)
            pages = loader.load()

            # 使用小批量处理文档
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=150
            )

            splits = []
            # 批量处理文档页面
            for i in range(0, len(pages), self.batch_size):
                batch = pages[i:i + self.batch_size]
                batch_splits = text_splitter.split_documents(batch)
                splits.extend(batch_splits)

                # 主动释放内存
                gc.collect()

            return splits

        except Exception as e:
            print(f"处理文件 {file_path} 时出错: {str(e)}")
            return []

    def create_vectorstore(self, persist_dir: str = "chroma_db"):
        """处理所有PDF文件并创建向量存储"""
        all_docs = []
        pdf_files = [f for f in os.listdir(self.pdf_dir) if f.endswith('.pdf')]

        print(f"找到 {len(pdf_files)} 个PDF文件")

        # 使用tqdm显示进度
        for pdf_file in tqdm(pdf_files, desc="处理PDF文件"):
            file_path = os.path.join(self.pdf_dir, pdf_file)
            docs = self.process_pdf(file_path)
            all_docs.extend(docs)
            print(f"- {pdf_file}: 处理了 {len(docs)} 个文档块")

        print(f"\n总共处理了 {len(all_docs)} 个文档块")

        # 创建向量存储
        vectorstore = Chroma.from_documents(
            documents=all_docs,
            embedding=self.embeddings,
            persist_directory=persist_dir
        )

        # 持久化存储
        vectorstore.persist()
        return vectorstore

# 4. 主执行流程
def main():
    # 挂载Google Drive

    # 设置路径
    VECTOR_STORE_PATH = "/content/drive/MyDrive/vector_store"  # 替换为向量存储路径

    print(f"开始处理: {datetime.now()}")

    # 创建处理器并处理文档
    processor = SimplePDFProcessor(
        pdf_dir=PDF_DIR,
        batch_size=5  # 可以根据文件大小和可用内存调整
    )

    # 处理文档并创建向量存储
    vectorstore = processor.create_vectorstore(VECTOR_STORE_PATH)

    print(f"处理完成: {datetime.now()}")
    return vectorstore

# 5. 运行处理
if __name__ == "__main__":
    vectorstore = main()

开始处理: 2025-06-10 09:05:30.903261


<ipython-input-16-3b7d96f13e17>:35: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

找到 2 个PDF文件


处理PDF文件:   0%|          | 0/2 [00:00<?, ?it/s]

- Manual_D3.pdf: 处理了 2793 个文档块
- pbapg.pdf: 处理了 1274 个文档块

总共处理了 4067 个文档块
处理完成: 2025-06-10 09:10:13.204513


<ipython-input-16-3b7d96f13e17>:93: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
## 8. 设置RAG Chain
# 创建记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
# 创建模板
rag_template = """Please use next context to answer the questions. If you do not know about the questions, just say you do not know, please do not make up the answers.Please try your best to use the context information.

context：{context}

question：{question}

Please answer with your understanding："""

PROMPT = PromptTemplate(
    template=rag_template,
    input_variables=["context", "question"]
)

# 创建对话链
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": PROMPT}
)

<ipython-input-17-2b24b53ddd3b>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [ ]:
## 9. 创建Gradio界面
def respond(message, history):
    response = chain({"question": message})
    return response['answer']

# 创建Gradio界面
demo = gr.ChatInterface(
    respond,
    title="RAG Documents Q&A System",
    description="Based on the Fine-tunning LLama model, the RAG system can answer the questions that related to uploaded documents."
)

# 启动界面
demo.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://570a88a3cdeb810ad9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
